# Patronus `Lynx` Hallucination Detection

This notebook will illustrates how to use `Lynx` from Patronus, a custom model for hallucination evaluation with the Weaviate Query Agent.

> This notebook is using patronus `0.1.3`

### Quick Patronus Setup Test

In [7]:
import os
import patronus
from patronus.evals import RemoteEvaluator

patronus.init(
    os.getenv("PATRONUS_API_KEY")
)

patronus_evaluator = RemoteEvaluator("lynx", "patronus:hallucination")
# See other built-in evaluators here - https://docs.patronus.ai/docs/evaluation_api/reference_guide

result = patronus_evaluator.evaluate(
    task_input="What is the largest animal in the world?",
    task_context=["The blue whale is the largest known animal on the planet.","In Dune by Frank Herbert, Sandworms are the largest animals - beware if you like spice!"],
    task_output="The giant sandworm.",
    gold_answer=""
)

print(result)

/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_86934/129833330.py:4: UserWarning: The Patronus SDK has already been initialized. Duplicate initialization attempts are ignored.
  patronus.init(


score=0.01 pass_=False text_output=None metadata={'positions': [[0, 19]], 'extra': None, 'confidence_interval': None} explanation='\'The context mentions that the blue whale is the largest known animal on the planet.\', \'The context also mentions that in the book Dune, Sandworms are the largest animals, but this is in a fictional context.\', "The answer \'The giant sandworm\' is not faithful to the context because it incorrectly identifies a fictional entity as the largest animal in the world, whereas the context clearly states that the blue whale is the largest known animal."' tags={} dataset_id=None dataset_sample_id=None evaluation_duration=datetime.timedelta(microseconds=986000) explanation_duration=datetime.timedelta(0)


### Import Weaviate Blogs to Weaviate

In [4]:
import os
import weaviate

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
)

In [3]:
import os
import tiktoken
import time

import weaviate
import weaviate.collections.classes.config as wvcc
from dotenv import load_dotenv
from weaviate.classes.init import AdditionalConfig, Timeout

load_dotenv()

local_blogs = []

main_folder_path = "./blog/"

for i, folder_name in enumerate(os.listdir(main_folder_path)):
    subfolder_path = os.path.join(main_folder_path, folder_name)
    if os.path.isdir(subfolder_path):
        index_file_path = os.path.join(subfolder_path, "index.mdx")
        if os.path.isfile(index_file_path):
            with open(index_file_path, "r", encoding="utf-8") as file:
                content = file.read()
                local_blogs.append(
                    {
                        "content": content,
                    }
                )

if weaviate_client.collections.exists("Blogs"):
    weaviate_client.collections.delete("Blogs")
blogs = weaviate_client.collections.create(
    name="Blogs",
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_weaviate(),
    properties=[
        wvcc.Property(name="content", data_type=wvcc.DataType.TEXT),
    ],
)

def chunk_text(text, max_tokens=300):
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(text)
    chunks = []
    
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = enc.decode(chunk_tokens)
        chunks.append(chunk_text)
    
    return chunks

chunked_blogs = []
for blog in local_blogs:
    chunks = chunk_text(blog["content"])
    for chunk in chunks:
        chunked_blogs.append({
            "content": chunk
        })

start_time = time.time()
with weaviate_client.batch.dynamic() as batch:
    for blog_chunk in chunked_blogs:
        batch.add_object(
            collection="Blogs",
            properties={
                "content": blog_chunk["content"],
            }
        )
end_time = time.time()
upload_time = end_time - start_time

print(f"Successfully imported {len(chunked_blogs)} blog chunks into Weaviate.")
print(f"Upload time: {upload_time:.2f} seconds")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/weaviate/collections/classes/config.py:1950: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for cls_field in self.model_fields:


Successfully imported 1463 blog chunks into Weaviate.
Upload time: 9.30 seconds


### Weaviate Query Agent

In [5]:
from weaviate.agents.query import QueryAgent
from weaviate.agents.utils import print_query_agent_response

qa = QueryAgent(
    client=weaviate_client, collections=["Blogs"]
)

response = qa.run("How does HNSW work?")
print_query_agent_response(response)

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ How does HNSW work?                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ HNSW (Hierarchical Navigable Small World) is a graph-based algorithm used for approximate nearest neighbor      │
│ search, which is highly effective for vector search operations. The core idea behind HNSW is to organize data   │
│ points into a hierarchical, multi-layered graph structure. This structure allows for speedy navigation and      │
│ searching within a dataset by balancing longer distances for fast navigation in the upper layers and shorter    │
│ distances for precise local searching in the lower layers.                                                      │
│                                                                                                                 │
│ In an HNSW graph, the top layers are sparse and contain only long-range connections, which enable rapid         │
│ traversal across the dataset. As a search query moves down through the hierarchy, the graph becomes denser,     │
│ allowing for finer-grained searches until the closest data points are found in the bottom layers.               │
│                                                                                                                 │
│ HNSW is beneficial for large-scale vector search databases, like Weaviate, where it supports fast, reliable     │
│ queries with high recall rates (>95%) while maintaining low latency. The algorithm's hierarchical structure is  │
│ key to its performance, as it allows decomposition of the search space, leading to efficient data traversal and │
│ retrieval.                                                                                                      │
│                                                                                                                 │
│ Additionally, HNSW can be enhanced with features such as real-time indexing, updates, and deletions through     │
│ CRUD operations, which is particularly useful for dynamic datasets that require frequent updates.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['HNSW working mechanism'],                                                                         │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Blogs'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='6d45f1f2-136f-4ada-9725-d463b72132de' collection='Blogs'                                          │
│  - object_id='bf57ffc0-e6fb-41ed-a724-4499306bd4e4' collection='Blogs'                                          │
│  - object_id='2727f48e-986a-4635-b967-6a4733b0da97' collection='Blogs'                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 8858 │
│ Output Tokens: │ 361  │
│ Total Tokens:  │ 9219 │
└────────────────┴──────┘

Total Time Taken: 11.06s

### `sources` from the Query Agent

The response from the Weaviate Query Agent contains a `sources` field that can be used to help users understand what influenced the final answer.

In [21]:
response.sources

[Source(object_id='6d45f1f2-136f-4ada-9725-d463b72132de', collection='Blogs'),
 Source(object_id='bf57ffc0-e6fb-41ed-a724-4499306bd4e4', collection='Blogs'),
 Source(object_id='2727f48e-986a-4635-b967-6a4733b0da97', collection='Blogs')]

In [14]:
from weaviate.classes.query import Filter

blogs_collection = weaviate_client.collections.get("Blogs")

source_uuids = [source.object_id for source in response.sources]
source_uuids

objects = blogs_collection.query.fetch_objects_by_ids(
    source_uuids
)

# Format the search results in a clean, numbered format
search_results = []
for i, o in enumerate(objects.objects, 1):
    content = o.properties.get('content', '')
    search_results.append(f"Search Result #{i}\n\n{content}\n")

# Join all results into a single string and print
formatted_results = "\n".join(search_results)
print(formatted_results)

Search Result #1

 pride ourselves on our research acumen and on providing state-of-the-art solutions. So we took time to explore these solutions to identify and evaluate the right building blocks for Weaviate's future.  Here we share some of our findings from this research.

## On the HNSW vs. Vamana comparison
As the first step to disk-based vector indexing, we decided to explore Vamana – the algorithm behind the DiskANN solution. Here are some key differences between Vamana and HNSW:

### Vamana indexing - in short:
* Build a random graph.
* Optimize the graph, so it only connects vectors close to each other.
* Modify the graph by removing some short connections and adding some long-range edges to speed up the traversal of the graph.

### HNSW indexing - in short:
* Build a hierarchy of layers to speed up the traversal of the nearest neighbor graph.
* In this graph, the top layers contain only long-range edges.
* The deeper the search traverses through the hierarchy, the shorter the

### Evaluate Query Agent Hallucination with Patronus `Lynx`

In [18]:
result = patronus_evaluator.evaluate(
    task_input="How does HNSW work?",
    task_context=[formatted_results],
    task_output=response.final_answer,
    gold_answer=""
)

print(result)

score=1.0 pass_=True text_output=None metadata={'positions': [[1095, 1159], [1344, 1351], [1224, 1231], [511, 576], [245, 509], [879, 901], [1047, 1093], [1233, 1270], [864, 877], [679, 703], [115, 243], [578, 677], [1161, 1222], [705, 862]], 'extra': None, 'confidence_interval': None} explanation='- The QUESTION asks about how HNSW works.\n- The CONTEXT provides detailed information about HNSW, including its structure and functionality.\n- The ANSWER summarizes the key points from the CONTEXT, explaining that HNSW is a graph-based algorithm for approximate nearest neighbor search.\n- The ANSWER mentions that HNSW organizes data points into a hierarchical, multi-layered graph structure, which allows for speedy navigation and searching.\n- The ANSWER also notes that the top layers contain only long-range connections for fast traversal, while the lower layers provide finer-grained searches for precise results.\n- The ANSWER highlights the benefits of HNSW, such as high recall rates and l

### Success!

In [22]:
print(result.score)
print(result.pass_)
print(result.explanation)

1.0
True
- The QUESTION asks about how HNSW works.
- The CONTEXT provides detailed information about HNSW, including its structure and functionality.
- The ANSWER summarizes the key points from the CONTEXT, explaining that HNSW is a graph-based algorithm for approximate nearest neighbor search.
- The ANSWER mentions that HNSW organizes data points into a hierarchical, multi-layered graph structure, which allows for speedy navigation and searching.
- The ANSWER also notes that the top layers contain only long-range connections for fast traversal, while the lower layers provide finer-grained searches for precise results.
- The ANSWER highlights the benefits of HNSW, such as high recall rates and low latency, making it suitable for large-scale vector search databases like Weaviate.
- The ANSWER also mentions the support for CRUD operations, which is useful for dynamic datasets.
- The ANSWER is faithful to the CONTEXT as it accurately reflects the information provided about HNSW, including